In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [4]:
# 뉴스 키워드 추출 예시 데이터
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라의 일론 머스크 CEO가 트위터를 인수한 후 대대적인 조직 개편을 단행했다. 직원 수를 대폭 줄이고 새로운 수익 모델을 도입하여 플랫폼의 지속가능성을 확보하려는 시도로 보인다.",
        "keywords": "테슬라, 일론머스크, 트위터"
    },
    {
        "news": "한국은행이 기준금리를 동결하기로 결정했다고 발표했다. 인플레이션 압력과 경제 성장률 둔화 사이에서 신중한 통화정책을 유지하겠다는 의지를 나타낸 것으로 분석된다.",
        "keywords": "한국은행, 기준금리, 인플레이션"
    }
]

# 1. 예시 프롬프트 (대화형)
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{news}"),
        ("ai", "키워드: {keywords}"),
    ]
)

# 2. Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


# 3. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])


In [5]:

chain = final_prompt | llm | StrOutputParser()

# 주어진 테스트 뉴스로 실행
test_news = """
제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.
"""

result = chain.invoke({"input": test_news})

print("뉴스 키워드 추출기 테스트 (ChatPromptTemplate 버전)")
print("=======================================================")
print("\n테스트 뉴스:")
print(test_news)
print("\n추출된 키워드:")
print(result)

뉴스 키워드 추출기 테스트 (ChatPromptTemplate 버전)

테스트 뉴스:

제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 
모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 
내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.


추출된 키워드:
키워드: 제미나이, 구글, AI
